# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# District Extracts & Calculations
district_df = school_data_complete.copy()                                              #- get df copy for district 
district_math_df = student_data.loc [(student_data["math_score"] >= 70),:]             #- get df for math scores 70 & above
district_read_df = student_data.loc [(student_data["reading_score"] >= 70),:]          #- get df for reading scores 70++

school_tot = school_data['school_name'].value_counts().sum()                           #- number of schools
student_tot = student_data.loc[:,"Student ID"].value_counts().sum()                    #- number of students using ID
budget_tot = school_data.loc [:,"budget"].sum()                                        #- total budget across schools

avg_math = student_data.loc [:,"math_score"].mean()                                    #- average math score
avg_read = student_data.loc [:,"reading_score"].mean()                                 #- average reading scores
avg_score = (avg_math + avg_read)/2                                                    #- calc overall passing rate
pass_math = district_math_df ["Student ID"].value_counts().sum()/student_tot * 100.00  #- count passing math studs
pass_read = district_read_df ["Student ID"].value_counts().sum()/student_tot * 100.00  #- count passing reading studs

# District Summary & Formatting
district_sum = pd.DataFrame({"Total Schools":[school_tot],
                             "Total Students":[student_tot],
                             "Total Budget":[budget_tot],
                             "Avg Math Score":[avg_math],
                             "Avg Reading Score":[avg_read],
                             "% Passing Math":[pass_math],
                             "% Passing Reading":[pass_read],
                             "% Overall Passing Rate":[avg_score]
                            })
district_sum["Total Students"] = district_sum["Total Students"].map("{:,}".format)     #- format studs
district_sum["Total Budget"] = district_sum["Total Budget"].map("${:,.2f}".format)     #- format budget

district_sum

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
# School Extracts, Groupings and Calculations

school_df = district_df.groupby(["school_name"])                      #- df by school
school_math_df = district_math_df.groupby(["school_name"])            #- df by school for math scores 70++
school_read_df = district_read_df.groupby(["school_name"])            #- df by school for reading scores 70++

school_type = school_df["type"].min()                                 #- use min to get type. there is only 1 per school
school_budget = school_df["budget"].min()                             #- use min to get budget. there is only 1 per school
school_size = school_df["size"].min()                                 #- use min to get size. there is only 1 size per sch
school_student_budget = pd.DataFrame({"school_student_budget":(school_budget / school_size)}) # calc budget per student

school_avg_math = school_df["math_score"].mean()                      #- calc mean for math score using df grouped by sch
school_avg_read = school_df["reading_score"].mean()                   #- cals mean for reading score per school
school_avg_score = pd.DataFrame({"overall_passing_rate":(school_avg_math + school_avg_read)/2})  #calc overall passing rte

school_math_size = school_math_df["Student ID"].count()               #- count of students with math scores 70++, per sch
school_read_size = school_read_df["Student ID"].count()               #- no. of students with reading scroes 70++
#school_size2 = school_df["Student ID"].count()

school_pass_math = pd.DataFrame({"passing_math":(school_math_size / school_size * 100)})
school_pass_read = pd.DataFrame({"passing_reading":(school_read_size / school_size * 100)})
school_pass_score = pd.DataFrame({"overall":((school_pass_math["passing_math"] + school_pass_read["passing_reading"]) /2)})


In [4]:
# School Summary & Formatting

school_sum = pd.merge(school_type, school_size, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_budget, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_student_budget, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_avg_math, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_avg_read, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_pass_math, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_pass_read, how="left", on=["school_name"])
school_sum = pd.merge(school_sum, school_pass_score, how="left", on=["school_name"])

school_sum2 = school_sum.copy()                         # create copy of school summary before reformatting below

school_sum["budget"] = school_sum["budget"].map("${:,.2f}".format)                      
school_sum["school_student_budget"] = school_sum["school_student_budget"].map("${:,.2f}".format)

school_sum = school_sum.rename(columns={"type":"School Type","size":"Total Students","budget":"Total School Budget",
                                        "school_student_budget":"Per Student Budget",
                                        "math_score":"Average Math Score","reading_score":"Average Reading Score",
                                        "passing_math":"% Passing Math","passing_reading":"% Passing Reading",
                                        "overall":"% Overall Passing Rate"
                                       })
school_sum

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
school_top = school_sum.sort_values(["% Overall Passing Rate"],ascending=False) #- sort school sum df in descending order
school_top.head(5)                                                              #- explicitly list first 5 (the default)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
school_bad = school_sum.sort_values(["% Overall Passing Rate"])         #- ascending is default
school_bad.head(5)                                                      #- explicitlt list first 5 (the default)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# Scores by Grade - extracts and mean calculations
grade_09_df = district_df.loc[district_df["grade"]=="9th",:]              #- collect 9th graders across districts
grade_10_df = district_df.loc[district_df["grade"]=="10th",:]             #- collect 10th graders across districts
grade_11_df = district_df.loc[district_df["grade"]=="11th",:]             #- collect 11th graders across districts
grade_12_df = district_df.loc[district_df["grade"]=="12th",:]             #- collect 12th graders across districts

grade_09_grp = grade_09_df.groupby(["school_name"])                       #- group 9th graders per school
grade_10_grp = grade_10_df.groupby(["school_name"])                       #- group 10th graders per school
grade_11_grp = grade_11_df.groupby(["school_name"])                       #- group 11th graders per school
grade_12_grp = grade_12_df.groupby(["school_name"])                       #- group 12th graders per school

grade_09 = grade_09_grp.mean()                                            #- mean 9th graders  :)
grade_10 = grade_10_grp.mean()                                            #- mean 10th graders :)
grade_11 = grade_11_grp.mean()                                            #- mean 11th graders :)
grade_12 = grade_12_grp.mean()                                            #- mean 12th graders :)

# Scores by Grade - Math                                                  #- create df for school x grade matrix, for math
grade_math_sum = pd.DataFrame({"9th":grade_09["math_score"],"10th":grade_10["math_score"],
                              "11th":grade_11["math_score"],"12th":grade_12["math_score"]})
grade_math_sum

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# Scores by Grade - Reading                                       #- create df for school x grade matrix, for reading
grade_read_sum = pd.DataFrame({"9th":grade_09["reading_score"],"10th":grade_10["reading_score"],
                              "11th":grade_11["reading_score"],"12th":grade_12["reading_score"]})
grade_read_sum

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# create spending summary using school summary df
# however, from here on, use copy of school summary df (this copy was taken above before any formatting)

spending_sum = pd.DataFrame({"Per Student Budget":school_sum2["school_student_budget"],
                             "Average Math Score":school_sum2["math_score"],
                             "Average Reading Score":school_sum2["reading_score"],
                             "% Passing Math":school_sum2["passing_math"],
                             "% Passing Reading":school_sum2["passing_reading"],
                             "% Overall Passing Rate":school_sum2["overall"]
                            })

In [10]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# create new column based on spending bins, then group by it
spending_sum["spending_range"] = pd.cut(school_sum2["school_student_budget"], spending_bins, labels=group_names)
spending_grp = spending_sum.groupby(["spending_range"])
spending_grp.mean()


,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
spending_range,,,,,,
<$585,581.000000,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,604.500000,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,635.166667,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,652.333333,76.997210,81.027843,66.164813,81.133951,73.649382


In [11]:
# ---------------------------------------------------------------------------------------------------------------------------------------------------
# Important: The above cell created the required homework results, but with extraneous column that could not be dropped easily at the groupby level.
# Alternative solution below applies groupby at each column separately, then brings all together into a df for the final report.
# I prefer the first solution since it is practical to include all columns, then report on the needed columns at the end.
# However, more research on using "apply" is required.
# ---------------------------------------------------------------------------------------------------------------------------------------------------

spending_avg_math = spending_sum.loc[:,["Average Math Score","spending_range"]].groupby(["spending_range"])
spending_avg_read = spending_sum.loc[:,["Average Reading Score","spending_range"]].groupby(["spending_range"])
spending_pass_math = spending_sum.loc[:,["% Passing Math","spending_range"]].groupby(["spending_range"])
spending_pass_read = spending_sum.loc[:,["% Passing Reading","spending_range"]].groupby(["spending_range"])
spending_overall = spending_sum.loc[:,["% Overall Passing Rate","spending_range"]].groupby(["spending_range"])

spending_rpt = pd.DataFrame({"Average Math Score":spending_avg_math.mean()["Average Math Score"],
                             "Average Reading Score":spending_avg_read.mean()["Average Reading Score"],
                             "% Passing Math":spending_pass_math.mean()["% Passing Math"],
                             "% Passing Reading":spending_pass_read.mean()["% Passing Reading"],
                             "% Overall Passing Rate":spending_overall.mean()["% Overall Passing Rate"],
                            })
spending_rpt

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
spending_range,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

spending_sum["size_range"] = pd.cut(school_sum2["size"], size_bins, labels=group_names)
size_grp = spending_sum.groupby(["size_range"])
size_grp.mean()


,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
size_range,,,,,,
Small (<1000),595.000,83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),605.600,83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),635.375,77.746417,81.344493,69.963361,82.766634,76.364998


In [13]:
size_sum = pd.DataFrame({"Per Student Budget":school_sum2["school_student_budget"],
                             "Average Math Score":school_sum2["math_score"],
                             "Average Reading Score":school_sum2["reading_score"],
                             "% Passing Math":school_sum2["passing_math"],
                             "% Passing Reading":school_sum2["passing_reading"],
                             "% Overall Passing Rate":school_sum2["overall"]
                            })

size_sum["size_range"] = pd.cut(school_sum2["size"], size_bins, labels=group_names)

size_avg_math = size_sum.loc[:,["Average Math Score","size_range"]].groupby(["size_range"])
size_avg_read = size_sum.loc[:,["Average Reading Score","size_range"]].groupby(["size_range"])
size_pass_math = size_sum.loc[:,["% Passing Math","size_range"]].groupby(["size_range"])
size_pass_read = size_sum.loc[:,["% Passing Reading","size_range"]].groupby(["size_range"])
size_overall = size_sum.loc[:,["% Overall Passing Rate","size_range"]].groupby(["size_range"])

size_rpt = pd.DataFrame({"Average Math Score":size_avg_math.mean()["Average Math Score"],
                             "Average Reading Score":size_avg_read.mean()["Average Reading Score"],
                             "% Passing Math":size_pass_math.mean()["% Passing Math"],
                             "% Passing Reading":size_pass_read.mean()["% Passing Reading"],
                             "% Overall Passing Rate":size_overall.mean()["% Overall Passing Rate"],
                            })
size_rpt

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
size_range,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
type_grp = school_sum2.groupby(["type"])
type_grp.mean()


,size,budget,school_student_budget,math_score,reading_score,passing_math,passing_reading,overall
type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,83.473852,83.896421,93.620830,96.586489,95.103660
District,3853.714286,2.478275e+06,643.571429,76.956733,80.966636,66.548453,80.799062,73.673757


In [15]:
type_sum = pd.DataFrame({"Per Student Budget":school_sum2["school_student_budget"],
                             "Average Math Score":school_sum2["math_score"],
                             "Average Reading Score":school_sum2["reading_score"],
                             "% Passing Math":school_sum2["passing_math"],
                             "% Passing Reading":school_sum2["passing_reading"],
                             "% Overall Passing Rate":school_sum2["overall"],
                             "School Type":school_sum2["type"]
                            })

type_avg_math = type_sum.loc[:,["Average Math Score","School Type"]].groupby(["School Type"])
type_avg_read = type_sum.loc[:,["Average Reading Score","School Type"]].groupby(["School Type"])
type_pass_math = type_sum.loc[:,["% Passing Math","School Type"]].groupby(["School Type"])
type_pass_read = type_sum.loc[:,["% Passing Reading","School Type"]].groupby(["School Type"])
type_overall = type_sum.loc[:,["% Overall Passing Rate","School Type"]].groupby(["School Type"])

type_rpt = pd.DataFrame({"Average Math Score":type_avg_math.mean()["Average Math Score"],
                             "Average Reading Score":type_avg_read.mean()["Average Reading Score"],
                             "% Passing Math":type_pass_math.mean()["% Passing Math"],
                             "% Passing Reading":type_pass_read.mean()["% Passing Reading"],
                             "% Overall Passing Rate":type_overall.mean()["% Overall Passing Rate"],
                            })
type_rpt

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Observable Trends

* Charter schools which requires less budget outperforms District schools, especially in math scores
* A budget can be kept at around $600 per student and keep the high passing marks, for as along as size is kept within small to medium range